# TF Regression Project

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census.

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

Dataset -  http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

** Import data **

In [1]:
import pandas as pd

In [2]:
housing = pd.read_csv('cal_housing_clean.csv')

In [3]:
housing.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [4]:
housing.describe().transpose()
 

,count,mean,std,min,25%,50%,75%,max
housingMedianAge,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
totalRooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
totalBedrooms,20640.0,537.898014,421.247906,1.0000,295.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
medianIncome,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
medianHouseValue,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


In [5]:
x_data = housing.drop(['medianHouseValue'],axis=1)

In [6]:
y_val = housing['medianHouseValue']

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(x_data,y_val,test_size=0.3,random_state=101)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. 

In [9]:
from sklearn.preprocessing import MinMaxScaler

In [10]:
scaler = MinMaxScaler()

In [11]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [12]:
X_train = pd.DataFrame(data=scaler.transform(X_train),columns = X_train.columns,index=X_train.index)
X_test = pd.DataFrame(data=scaler.transform(X_test),columns = X_test.columns,index=X_test.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator **

In [13]:
housing.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [14]:
import tensorflow as tf

C:\Users\Rahman\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


** Create continuous feature_columns for the continuous values using numeric_column **

In [15]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
pop = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

In [16]:
feat_cols = [ age,rooms,bedrooms,pop,households,income]

** Create the input function for the estimator object. **

In [17]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train ,batch_size=10,num_epochs=1000,
                                            shuffle=True)

** Create the estimator model, Use a DNNRegressor **
** Create 3 layer and 6 neoron for each layer **

In [18]:
model = tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Rahman\\AppData\\Local\\Temp\\tmp7viio4i1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000021E302779E8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


#### ** Train the model for 10000 steps. **

In [19]:
model.train(input_fn=input_func,steps=10000)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\Rahman\AppData\Local\Temp\tmp7viio4i1\model.ckpt.
INFO:tensorflow:loss = 585524640000.0, step = 1
INFO:tensorflow:global_step/sec: 275.963
INFO:tensorflow:loss = 659495300000.0, step = 101 (0.362 sec)
INFO:tensorflow:global_step/sec: 351.932
INFO:tensorflow:loss = 695286240000.0, step = 201 (0.301 sec)
INFO:tensorflow:global_step/sec: 337.761
INFO:tensorflow:loss = 563958800000.0, step = 301 (0.279 sec)
INFO:tensorflow:global_step/sec: 340.471
INFO:tensor

INFO:tensorflow:global_step/sec: 295.395
INFO:tensorflow:loss = 100004930000.0, step = 6701 (0.352 sec)
INFO:tensorflow:global_step/sec: 301.249
INFO:tensorflow:loss = 129058800000.0, step = 6801 (0.343 sec)
INFO:tensorflow:global_step/sec: 302.87
INFO:tensorflow:loss = 51665120000.0, step = 6901 (0.326 sec)
INFO:tensorflow:global_step/sec: 280.532
INFO:tensorflow:loss = 111959880000.0, step = 7001 (0.335 sec)
INFO:tensorflow:global_step/sec: 292.424
INFO:tensorflow:loss = 142735540000.0, step = 7101 (0.348 sec)
INFO:tensorflow:global_step/sec: 292.11
INFO:tensorflow:loss = 218421600000.0, step = 7201 (0.339 sec)
INFO:tensorflow:global_step/sec: 314.713
INFO:tensorflow:loss = 66542227000.0, step = 7301 (0.332 sec)
INFO:tensorflow:global_step/sec: 325.882
INFO:tensorflow:loss = 213172830000.0, step = 7401 (0.310 sec)
INFO:tensorflow:global_step/sec: 289.187
INFO:tensorflow:loss = 104784680000.0, step = 7501 (0.332 sec)
INFO:tensorflow:global_step/sec: 303.974
INFO:tensorflow:loss = 2084

** Create prediction input function then use .predict method off estimator model to create a list or predictions on test data. **

In [20]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [21]:
pred_gen = model.predict(predict_input_func)

In [22]:
predictions = list(pred_gen)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Rahman\AppData\Local\Temp\tmp7viio4i1\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


### ** Calculate the RMSE. **

In [23]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])

In [29]:
final_preds[:10]

[array([228244.97], dtype=float32),
 array([271712.47], dtype=float32),
 array([223316.34], dtype=float32),
 array([180512.23], dtype=float32),
 array([246377.16], dtype=float32),
 array([202837.62], dtype=float32),
 array([232586.06], dtype=float32),
 array([208021.25], dtype=float32),
 array([208304.3], dtype=float32),
 array([167561.], dtype=float32)]

In [30]:
from sklearn.metrics import mean_squared_error

In [31]:
print('MSE : {}'.format(mean_squared_error(y_test,final_preds)**0.5))

MSE : 104999.69308056065
